In [1]:
import pickle
import numpy as np
import matplotlib.pyplot as plt
import timm
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as T
from torchvision import models
from torchsummary import summary
import torch.nn as nn
import torch.nn.functional as F
import cv2
from utils import preprocess_image

# update the requirements.txt file
# !pip freeze > requirements.txt

/home/pradeep/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


In [8]:
weights = models.segmentation.DeepLabV3_ResNet101_Weights.COCO_WITH_VOC_LABELS_V1
weights_pretrained = models.ResNet101_Weights.IMAGENET1K_V2
model = models.segmentation.deeplabv3.deeplabv3_resnet101(weights=weights, weights_backbone=weights_pretrained)
# print(model)

In [5]:
image = cv2.imread("Data/1.jpg")
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
image = preprocess_image(image)

model.eval()
output = model(image.unsqueeze(0))

print

In [6]:
output = output["out"]
output.shape

torch.Size([1, 21, 720, 930])

In [5]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)

In [6]:
pkl_path = "segmented_images.pkl"
with open(pkl_path, "rb") as file:
    data_dict = pickle.load(file)

data_dict

In [27]:
for file_name, label in data_dict.items():
    image = cv2.imread(file_name)
    image = preprocess_image(image)
    model.eval()
    output = model(image.unsqueeze(0))
    output = output["out"]
    print(output.max(), output.min())
    break

In [22]:
loss_list = []

model.train()
for file_name, label in data_dict.items(): 

    image = cv2.imread(file_name)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    image = preprocess_image(image)
    model.eval()
    output = model(image.unsqueeze(0))

    output = output["out"]

    # train with ground truth as label
    label = torch.tensor(label)
    label = label.unsqueeze(0)

    # print(output.shape, label.shape)
    loss = F.cross_entropy(output, label)
    print(loss.item())
    loss_list.append(loss.item())

    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    

nan


KeyboardInterrupt: 